In [ ]:
%%capture
!pip install openai

In [ ]:
%%capture
!unzip /content/openstax_probing_dataset_polish.zip
!unzip /content/openstax_probing_dataset_spanish.zip

In [ ]:
import hashlib

def md5_hash(string):
    string_bytes = string.encode('utf-8')
    md5_hash = hashlib.md5()
    md5_hash.update(string_bytes)
    md5_hash_hex = md5_hash.hexdigest()
    return md5_hash_hex

In [ ]:
import os
import json
from tqdm import tqdm

directory = '/content/content/Spanish_v5'

id_to_json_line = {}

unique_ids = set()

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r') as file:
            data = json.load(file)

            for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  if unique_id not in unique_ids:
                    unique_ids.add(unique_id)

                    id_to_json_line[unique_id] = {"custom_id": "gpt-4o-mini_spanish_" + unique_id,
                            "method": "POST",
                            "url": "/v1/chat/completions",
                            "body": {"model": "gpt-4o-mini",
                                    "messages": [{"role": "system",
                                                  "content": "You are a helpful, respectful and honest assistant. Your answers should be crisp, short and not repetitive. Here are 5 examples: Example 1: Prompt: Francesco Bartolomeo Conti was born in [MASK]. Answer: Florence. Example 2: Prompt: English bulldog is a subclass of [MASK]. Answer: dog. Example 3: Prompt: The official language of Mauritius is [MASK]. Answer: English. Example 4: Prompt: Nicotine binds to [MASK]. Answer: CHRNA4. Example 5: Prompt: Hepatitis has symptoms such as [MASK]. Answer: Abdominal pain."},
                                                    {"role": "user", "content": f"Prompt: {prompt['prompt']}"}],"max_tokens": 1000}}


directory = '/content/content/Polish_v5'
for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r') as file:
            data = json.load(file)

            for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  if unique_id not in unique_ids:
                    unique_ids.add(unique_id)

                    id_to_json_line[unique_id] = {"custom_id": "gpt-4o-mini_polish_" + unique_id,
                            "method": "POST",
                            "url": "/v1/chat/completions",
                            "body": {"model": "gpt-4o-mini",
                                    "messages": [{"role": "system",
                                                  "content": "You are a helpful, respectful and honest assistant. Your answers should be crisp, short and not repetitive. Here are 5 examples: Example 1: Prompt: Francesco Bartolomeo Conti was born in [MASK]. Answer: Florence. Example 2: Prompt: English bulldog is a subclass of [MASK]. Answer: dog. Example 3: Prompt: The official language of Mauritius is [MASK]. Answer: English. Example 4: Prompt: Nicotine binds to [MASK]. Answer: CHRNA4. Example 5: Prompt: Hepatitis has symptoms such as [MASK]. Answer: Abdominal pain."},
                                                    {"role": "user", "content": f"Prompt: {prompt['prompt']}"}],"max_tokens": 1000}}



100%|██████████| 6/6 [00:00<00:00, 43.23it/s]


In [ ]:
len(id_to_json_line)

16561

In [ ]:
import json

with open("gpt-4o-mini-multilingual-full.json", 'w') as f:
  for k in id_to_json_line.keys():
    f.write(json.dumps(id_to_json_line[k]) + '\n')

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
file_path = "gpt-4o-mini-multilingual-full.json"

batch_input_file = client.files.create(
  file=open(file_path, "rb"),
  purpose="batch"
)

batch_input_file_id = batch_input_file.id


client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={

      "description": "gpt-4o-mini-multilingual-full"
    }
)

Batch(id='batch_670d951005e0819094fddfad4502fd5a', completion_window='24h', created_at=1728943376, endpoint='/v1/chat/completions', input_file_id='file-rB3bufM9q8XEgaN73v8gzuAf', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1729029776, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o-mini-multilingual-full'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
import json

def split_jsonl_file(file_path, num_splits=5):
    with open(file_path, 'r') as infile:
        lines = infile.readlines()

    total_lines = len(lines)
    lines_per_split = total_lines // num_splits
    remainder = total_lines % num_splits

    start = 0
    for i in range(num_splits):
        end = start + lines_per_split + (1 if i < remainder else 0)

        split_file_path = f'gpt-4-turbo-multilingual-split_{i+1}.jsonl'
        with open(split_file_path, 'w') as outfile:
            outfile.writelines([l.replace('gpt-4o-mini', 'gpt-4-turbo-2024-04-09') for l in lines[start:end]])

        print(f"Split {i+1} written to {split_file_path}")

        start = end


In [ ]:

split_jsonl_file('/content/gpt-4o-mini-multilingual-full.json', num_splits=5)

Split 1 written to gpt-4-turbo-multilingual-split_1.jsonl
Split 2 written to gpt-4-turbo-multilingual-split_2.jsonl
Split 3 written to gpt-4-turbo-multilingual-split_3.jsonl
Split 4 written to gpt-4-turbo-multilingual-split_4.jsonl
Split 5 written to gpt-4-turbo-multilingual-split_5.jsonl


In [ ]:
file_path = "/content/gpt-4-turbo-multilingual-split_5.jsonl"

batch_input_file = client.files.create(
  file=open(file_path, "rb"),
  purpose="batch"
)

batch_input_file_id = batch_input_file.id


client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={

      "description": "gpt-4-turbo-multilingual-split_5"
    }
)

Batch(id='batch_670dbbdb06108190bdecc247db191eef', completion_window='24h', created_at=1728953307, endpoint='/v1/chat/completions', input_file_id='file-iBbQCiF1S8TfY1yQBUUMOMLs', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1729039707, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4-turbo-multilingual-split_5'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
data_dicts = []
statuses = []
metadatum = []

for data in client.batches.list():
  data_dict = data.to_dict()

  if data_dict['status'] not in ['failed', 'cancelled', 'cancelling']:
    data_dicts.append(data_dict)
    statuses.append(data_dict['status'])
    metadatum.append(data_dict['metadata'])

batches_that_have_run = list(set([m['description'][len("gpt-4o-mini-multilingual") + 1:]+'.jsonl' for m in metadatum if "gpt-4o-mini-multilingual" in m['description']]))

KeyboardInterrupt: 

In [ ]:
texts =[]

for data in client.batches.list():
  data_dict = data.to_dict()
  if data_dict['status'] not in ['failed', 'cancelled', 'cancelling'] and 'gpt-4o-mini-multilingual' in data_dict['metadata']['description']:

    file_response = client.files.content(data_dict['output_file_id'])

    texts.append(file_response.text)

In [ ]:
len(texts)

1

In [ ]:
from tqdm import tqdm
import json

id_to_pred_gpt_4o_mini = {}

for text in tqdm(texts):
  lines = text.split('\n')

  for line in lines[:-1]:
    x = json.loads(line)
    id_to_pred_gpt_4o_mini[x['custom_id'].split('_')[-1]] = x['response']['body']['choices'][0]['message']['content']

100%|██████████| 1/1 [00:00<00:00,  4.85it/s]


In [ ]:
len(id_to_pred_gpt_4o_mini)

16561

In [ ]:
import hashlib

def md5_hash(string):
    string_bytes = string.encode('utf-8')
    md5_hash = hashlib.md5()
    md5_hash.update(string_bytes)
    md5_hash_hex = md5_hash.hexdigest()
    return md5_hash_hex

# GPT-4o-mini

In [ ]:
import os

def best_subspan_score(label, completion):
  if label.lower() in completion.lower():
    return 1
  else:
    return 0


directory = "/content/content/Spanish_v5"


total_pg = []
total_st = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        scores_pg = []
        scores_st = []
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  pred = id_to_pred_gpt_4o_mini[unique_id]

                  score = best_subspan_score(prompt['label'], pred)

                  assert level in ["paragraph_level", "sentence_level"]
                  if level == "paragraph_level":
                    scores_pg.append(score)
                  else:
                    scores_st.append(score)
        print(f"{filename}: pg_level: {sum(scores_pg)/len(scores_pg)}, st_level: {sum(scores_st)/len(scores_st)}")
        total_pg.extend(scores_pg)
        total_st.extend(scores_st)


print("GPT-4o-mini Spanish")


print()
print(f"Spanish total: pg_level: {sum(total_pg)/len(total_pg)}, st_level: {sum(total_st)/len(total_st)}")



100%|██████████| 10/10 [00:00<00:00, 109.42it/s]

prec%C3%A1lculo-2ed.json: pg_level: 0.3498312710911136, st_level: 0.2576419213973799
c%C3%A1lculo-volumen-2.json: pg_level: 0.26666666666666666, st_level: 0.1717171717171717
introducci%C3%B3n-estad%C3%ADstica-empresarial.json: pg_level: 0.41, st_level: 0.36231884057971014
qu%C3%ADmica-2ed.json: pg_level: 0.5104166666666666, st_level: 0.3923739237392374
c%C3%A1lculo-volumen-3.json: pg_level: 0.25609756097560976, st_level: 0.20647773279352227
f%C3%ADsica-universitaria-volumen-3.json: pg_level: 0.4190800681431005, st_level: 0.3224043715846995
c%C3%A1lculo-volumen-1.json: pg_level: 0.3249097472924188, st_level: 0.2579185520361991
f%C3%ADsica-universitaria-volumen-2.json: pg_level: 0.4333958724202627, st_level: 0.29471544715447157
f%C3%ADsica-universitaria-volumen-1.json: pg_level: 0.39065108514190316, st_level: 0.2730496453900709
introducci%C3%B3n-estad%C3%ADstica.json: pg_level: 0.3739495798319328, st_level: 0.32460732984293195
GPT-4o-mini Spanish

Spanish total: pg_level: 0.3962637362637

In [ ]:
import os

def best_subspan_score(label, completion):
  if label.lower() in completion.lower():
    return 1
  else:
    return 0


directory = "/content/content/Polish_v5"

total_pg = []
total_st = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        scores_pg = []
        scores_st = []
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  pred = id_to_pred_gpt_4o_mini[unique_id]

                  score = best_subspan_score(prompt['label'], pred)

                  assert level in ["paragraph_level", "sentence_level"]
                  if level == "paragraph_level":
                    scores_pg.append(score)
                  else:
                    scores_st.append(score)
        print(f"{filename}: pg_level: {sum(scores_pg)/len(scores_pg)}, st_level: {sum(scores_st)/len(scores_st)}")
        total_pg.extend(scores_pg)
        total_st.extend(scores_st)


print("GPT-4o-mini Polish")


print()
print(f"Spanish total: pg_level: {sum(total_pg)/len(total_pg)}, st_level: {sum(total_st)/len(total_st)}")



100%|██████████| 6/6 [00:00<00:00, 53.27it/s]

fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-3.json: pg_level: 0.23283082077051925, st_level: 0.1903914590747331
psychologia-polska.json: pg_level: 0.3238993710691824, st_level: 0.2001361470388019
fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-2.json: pg_level: 0.2578947368421053, st_level: 0.16977611940298507
mikroekonomia-podstawy.json: pg_level: 0.30183727034120733, st_level: 0.1856763925729443
fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-1.json: pg_level: 0.21833333333333332, st_level: 0.14564831261101244
makroekonomia-podstawy.json: pg_level: 0.28634361233480177, st_level: 0.20089285714285715
GPT-4o-mini Polish

Spanish total: pg_level: 0.28077290076335876, st_level: 0.18558786346396966


# GPT-4o

In [ ]:
texts =[]

for data in client.batches.list():
  data_dict = data.to_dict()
  if data_dict['status'] not in ['failed', 'cancelled', 'cancelling'] and 'gpt-4o-multilingual' in data_dict['metadata']['description']:

    file_response = client.files.content(data_dict['output_file_id'])

    texts.append(file_response.text)

In [ ]:
from tqdm import tqdm
import json

id_to_pred_gpt_4o = {}

for text in tqdm(texts):
  lines = text.split('\n')

  for line in lines[:-1]:
    x = json.loads(line)
    id_to_pred_gpt_4o[x['custom_id'].split('_')[-1]] = x['response']['body']['choices'][0]['message']['content']

100%|██████████| 6/6 [00:00<00:00, 14.11it/s]


In [ ]:
directory = "/content/content/Spanish_v5"

total_pg = []
total_st = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        scores_pg = []
        scores_st = []
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  pred = id_to_pred_gpt_4o[unique_id]

                  score = best_subspan_score(prompt['label'], pred)

                  assert level in ["paragraph_level", "sentence_level"]
                  if level == "paragraph_level":
                    scores_pg.append(score)
                  else:
                    scores_st.append(score)
        print(f"{filename}: pg_level: {sum(scores_pg)/len(scores_pg)}, st_level: {sum(scores_st)/len(scores_st)}")
        total_pg.extend(scores_pg)
        total_st.extend(scores_st)


print("GPT-4o Spanish")


print()
print(f"Spanish total: pg_level: {sum(total_pg)/len(total_pg)}, st_level: {sum(total_st)/len(total_st)}")

100%|██████████| 10/10 [00:00<00:00, 81.14it/s]

prec%C3%A1lculo-2ed.json: pg_level: 0.36332958380202474, st_level: 0.29112081513828236
c%C3%A1lculo-volumen-2.json: pg_level: 0.22962962962962963, st_level: 0.21212121212121213
introducci%C3%B3n-estad%C3%ADstica-empresarial.json: pg_level: 0.42, st_level: 0.34782608695652173
qu%C3%ADmica-2ed.json: pg_level: 0.5601851851851852, st_level: 0.46494464944649444
c%C3%A1lculo-volumen-3.json: pg_level: 0.3201219512195122, st_level: 0.27530364372469635
f%C3%ADsica-universitaria-volumen-3.json: pg_level: 0.4463373083475298, st_level: 0.36429872495446264
c%C3%A1lculo-volumen-1.json: pg_level: 0.38267148014440433, st_level: 0.27149321266968324
f%C3%ADsica-universitaria-volumen-2.json: pg_level: 0.4709193245778612, st_level: 0.3617886178861789
f%C3%ADsica-universitaria-volumen-1.json: pg_level: 0.41569282136894825, st_level: 0.30141843971631205
introducci%C3%B3n-estad%C3%ADstica.json: pg_level: 0.3739495798319328, st_level: 0.3036649214659686
GPT-4o Spanish

Spanish total: pg_level: 0.4268131868131

In [ ]:
directory = "/content/content/Polish_v5"

total_pg = []
total_st = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        scores_pg = []
        scores_st = []
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  pred = id_to_pred_gpt_4o[unique_id]

                  score = best_subspan_score(prompt['label'], pred)

                  assert level in ["paragraph_level", "sentence_level"]
                  if level == "paragraph_level":
                    scores_pg.append(score)
                  else:
                    scores_st.append(score)
        print(f"{filename}: pg_level: {sum(scores_pg)/len(scores_pg)}, st_level: {sum(scores_st)/len(scores_st)}")
        total_pg.extend(scores_pg)
        total_st.extend(scores_st)


print("GPT-4o Polish")


print()
print(f"Spanish total: pg_level: {sum(total_pg)/len(total_pg)}, st_level: {sum(total_st)/len(total_st)}")

  0%|          | 0/6 [00:00<?, ?it/s]

fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-3.json: pg_level: 0.39195979899497485, st_level: 0.30782918149466193


 33%|███▎      | 2/6 [00:00<00:00, 10.19it/s]

psychologia-polska.json: pg_level: 0.4276729559748428, st_level: 0.30428863172226006
fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-2.json: pg_level: 0.3526315789473684, st_level: 0.2574626865671642


100%|██████████| 6/6 [00:00<00:00, 16.12it/s]

mikroekonomia-podstawy.json: pg_level: 0.3569553805774278, st_level: 0.26525198938992045
fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-1.json: pg_level: 0.32666666666666666, st_level: 0.2202486678507993
makroekonomia-podstawy.json: pg_level: 0.3502202643171806, st_level: 0.25669642857142855
GPT-4o Polish

Spanish total: pg_level: 0.38311068702290074, st_level: 0.27737041719342603


# GPT-4-turbo

In [ ]:
texts =[]

for data in client.batches.list():
  data_dict = data.to_dict()
  if data_dict['status'] not in ['failed', 'cancelled', 'cancelling'] and 'gpt-4-turbo-multilingual' in data_dict['metadata']['description']:

    file_response = client.files.content(data_dict['output_file_id'])

    texts.append(file_response.text)

In [ ]:
print(len(texts))

4


In [ ]:
texts =[]

for data in client.batches.list():
  data_dict = data.to_dict()
  if data_dict['id'] in ['batch_670d9e926f5c81909e9f825ff353f288', 'batch_670da35951688190a9fc100a07d1d825', 'batch_670daad8a3648190ae6813753abe529a', 'batch_670db2da674c81909e3a91d9f9520e90', 'batch_670dbbdb06108190bdecc247db191eef']:

    file_response = client.files.content(data_dict['output_file_id'])

    texts.append(file_response.text)

In [ ]:
print(len(texts))

5


In [ ]:
from tqdm import tqdm
import json

id_to_pred_gpt_4_turbo = {}

for text in tqdm(texts):
  lines = text.split('\n')

  for line in lines[:-1]:
    x = json.loads(line)
    id_to_pred_gpt_4_turbo[x['custom_id'].split('_')[-1]] = x['response']['body']['choices'][0]['message']['content']

100%|██████████| 5/5 [00:00<00:00, 24.34it/s]


In [ ]:
for text in tqdm(texts):
  lines = text.split('\n')

  for line in lines[:-1]:
    x = json.loads(line)
    if x['custom_id'].split('_')[-1] not in id_to_pred_gpt_4_turbo.keys():
      id_to_pred_gpt_4_turbo[x['custom_id'].split('_')[-1]] = x['response']['body']['choices'][0]['message']['content']

100%|██████████| 5/5 [00:00<00:00, 26.39it/s]


In [ ]:
directory = "/content/content/Spanish_v5"

total_pg = []
total_st = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        scores_pg = []
        scores_st = []
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  pred = id_to_pred_gpt_4_turbo[unique_id]

                  score = best_subspan_score(prompt['label'], pred)

                  assert level in ["paragraph_level", "sentence_level"]
                  if level == "paragraph_level":
                    scores_pg.append(score)
                  else:
                    scores_st.append(score)
        print(f"{filename}: pg_level: {sum(scores_pg)/len(scores_pg)}, st_level: {sum(scores_st)/len(scores_st)}")
        total_pg.extend(scores_pg)
        total_st.extend(scores_st)


print("GPT-4-turbo Spanish")


print()
print(f"Spanish total: pg_level: {sum(total_pg)/len(total_pg)}, st_level: {sum(total_st)/len(total_st)}")

100%|██████████| 10/10 [00:00<00:00, 108.20it/s]

prec%C3%A1lculo-2ed.json: pg_level: 0.3734533183352081, st_level: 0.33478893740902477
c%C3%A1lculo-volumen-2.json: pg_level: 0.26666666666666666, st_level: 0.24242424242424243
introducci%C3%B3n-estad%C3%ADstica-empresarial.json: pg_level: 0.42, st_level: 0.42028985507246375
qu%C3%ADmica-2ed.json: pg_level: 0.5648148148148148, st_level: 0.46371463714637146
c%C3%A1lculo-volumen-3.json: pg_level: 0.3384146341463415, st_level: 0.2550607287449393
f%C3%ADsica-universitaria-volumen-3.json: pg_level: 0.4616695059625213, st_level: 0.3806921675774135
c%C3%A1lculo-volumen-1.json: pg_level: 0.37906137184115524, st_level: 0.29411764705882354
f%C3%ADsica-universitaria-volumen-2.json: pg_level: 0.46716697936210133, st_level: 0.3678861788617886
f%C3%ADsica-universitaria-volumen-1.json: pg_level: 0.4040066777963272, st_level: 0.3067375886524823
introducci%C3%B3n-estad%C3%ADstica.json: pg_level: 0.42016806722689076, st_level: 0.3036649214659686
GPT-4-turbo Spanish

Spanish total: pg_level: 0.43428571428

In [ ]:
directory = "/content/content/Polish_v5"

total_pg = []
total_st = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        scores_pg = []
        scores_st = []
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  pred = id_to_pred_gpt_4_turbo[unique_id]

                  score = best_subspan_score(prompt['label'], pred)

                  assert level in ["paragraph_level", "sentence_level"]
                  if level == "paragraph_level":
                    scores_pg.append(score)
                  else:
                    scores_st.append(score)
        print(f"{filename}: pg_level: {sum(scores_pg)/len(scores_pg)}, st_level: {sum(scores_st)/len(scores_st)}")
        total_pg.extend(scores_pg)
        total_st.extend(scores_st)


print("GPT-4-turbo Polish")


print()
print(f"Spanish total: pg_level: {sum(total_pg)/len(total_pg)}, st_level: {sum(total_st)/len(total_st)}")

100%|██████████| 6/6 [00:00<00:00, 35.82it/s]

fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-3.json: pg_level: 0.3065326633165829, st_level: 0.26868327402135234
psychologia-polska.json: pg_level: 0.32767295597484275, st_level: 0.23281143635125937
fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-2.json: pg_level: 0.30701754385964913, st_level: 0.21828358208955223
mikroekonomia-podstawy.json: pg_level: 0.33070866141732286, st_level: 0.22015915119363394
fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-1.json: pg_level: 0.295, st_level: 0.19715808170515098
makroekonomia-podstawy.json: pg_level: 0.28634361233480177, st_level: 0.22321428571428573
GPT-4-turbo Polish

Spanish total: pg_level: 0.31297709923664124, st_level: 0.22857142857142856


## Llama 3.1

In [ ]:
with open('/content/id_to_pred_spanish.json', 'r') as f:
  id_to_pred_llama_spanish = json.load(f)

In [ ]:
directory = "/content/content/Spanish_v5"

total_pg = []
total_st = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        scores_pg = []
        scores_st = []
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  pred = id_to_pred_llama_spanish[unique_id]

                  score = best_subspan_score(prompt['label'], pred)

                  assert level in ["paragraph_level", "sentence_level"]
                  if level == "paragraph_level":
                    scores_pg.append(score)
                  else:
                    scores_st.append(score)
        print(f"{filename}: pg_level: {sum(scores_pg)/len(scores_pg)}, st_level: {sum(scores_st)/len(scores_st)}")
        total_pg.extend(scores_pg)
        total_st.extend(scores_st)


print("Llama 3.1 Spanish")


print()
print(f"Spanish total: pg_level: {sum(total_pg)/len(total_pg)}, st_level: {sum(total_st)/len(total_st)}")

100%|██████████| 10/10 [00:00<00:00, 108.16it/s]

prec%C3%A1lculo-2ed.json: pg_level: 0.4881889763779528, st_level: 0.43377001455604075
c%C3%A1lculo-volumen-2.json: pg_level: 0.362962962962963, st_level: 0.2727272727272727
introducci%C3%B3n-estad%C3%ADstica-empresarial.json: pg_level: 0.57, st_level: 0.4782608695652174
qu%C3%ADmica-2ed.json: pg_level: 0.6354166666666666, st_level: 0.5313653136531366
c%C3%A1lculo-volumen-3.json: pg_level: 0.4146341463414634, st_level: 0.3157894736842105
f%C3%ADsica-universitaria-volumen-3.json: pg_level: 0.5178875638841567, st_level: 0.4426229508196721
c%C3%A1lculo-volumen-1.json: pg_level: 0.4657039711191336, st_level: 0.3031674208144796
f%C3%ADsica-universitaria-volumen-2.json: pg_level: 0.551594746716698, st_level: 0.4491869918699187
f%C3%ADsica-universitaria-volumen-1.json: pg_level: 0.44741235392320533, st_level: 0.3475177304964539
introducci%C3%B3n-estad%C3%ADstica.json: pg_level: 0.49159663865546216, st_level: 0.3717277486910995
Llama 3.1 Spanish

Spanish total: pg_level: 0.5136263736263736, st_

In [ ]:
with open('/content/id_to_pred_polish.json', 'r') as f:
  id_to_pred_llama_polish = json.load(f)

In [ ]:
directory = "/content/content/Polish_v5"

total_pg = []
total_st = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        scores_pg = []
        scores_st = []
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  pred = id_to_pred_llama_polish[unique_id]

                  score = best_subspan_score(prompt['label'], pred)

                  assert level in ["paragraph_level", "sentence_level"]
                  if level == "paragraph_level":
                    scores_pg.append(score)
                  else:
                    scores_st.append(score)
        print(f"{filename}: pg_level: {sum(scores_pg)/len(scores_pg)}, st_level: {sum(scores_st)/len(scores_st)}")
        total_pg.extend(scores_pg)
        total_st.extend(scores_st)


print("Llama 3.1 Polish")


print()
print(f"Polish total: pg_level: {sum(total_pg)/len(total_pg)}, st_level: {sum(total_st)/len(total_st)}")

100%|██████████| 6/6 [00:00<00:00, 27.10it/s]

fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-3.json: pg_level: 0.4204355108877722, st_level: 0.30782918149466193
psychologia-polska.json: pg_level: 0.4, st_level: 0.28727025187202176
fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-2.json: pg_level: 0.4, st_level: 0.292910447761194
mikroekonomia-podstawy.json: pg_level: 0.41732283464566927, st_level: 0.2493368700265252
fizyka-dla-szk%C3%B3%C5%82-wy%C5%BCszych-tom-1.json: pg_level: 0.3616666666666667, st_level: 0.23268206039076378
makroekonomia-podstawy.json: pg_level: 0.3788546255506608, st_level: 0.2700892857142857
Llama 3.1 Polish

Polish total: pg_level: 0.3967080152671756, st_level: 0.27762326169405815
